In [1]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import LSTM, Activation, Dropout, Embedding, Input
from keras import regularizers
from keras.optimizers import Nadam, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization

import json

import os

import math

import pandas as pd
import numpy as np

def import_log(filepath):
    df = pd.read_csv(filepath)
    return(df.values.tolist())

def remove_nan(lists):
    newlists = []
    for tr in lists:
        newlists.append([int(x) for x in tr if str(x) != 'nan'])
    return(newlists)

def create_XY_prefix(log, mappingsize, prefixlen):
    X = []
    Y = []
    for i in range(0, len(log)):
        for k in range(1, len(log[i])):
            X.append(log[i][max(0, k-prefixlen):k]) #get the prefix of 'encoded' activities
            y = [0] *(mappingsize)
            y[int(log[i][k])-1] = 1
            Y.append(y)        
    X = keras.preprocessing.sequence.pad_sequences(X, maxlen=prefixlen, padding='pre')
    return(np.array(X), np.array(Y))

def get_model_onedirectional(maxlen, num_chars, embedding, n_layers, lstmsize, dropout, l1, l2):
    emb_size = math.ceil(num_chars**0.25)
    model = Sequential()
    if embedding == True: #if you use embedding layer input should be number encoded
        model.add(Input(shape=(maxlen, )))
        model.add(Embedding(input_dim=num_chars+1, output_dim=emb_size))
    else:
        model.add(Input(shape=(maxlen, num_chars))) #If you don't use an embedding layer input should be one-hot-encoded
    model.add(LSTM(lstmsize,
                   kernel_initializer='glorot_uniform',
                   return_sequences=(n_layers != 1),
                   kernel_regularizer=regularizers.l1_l2(l1,l2),
                   recurrent_regularizer=regularizers.l1_l2(l1,l2),
                   input_shape=(maxlen, num_chars)))
    model.add(Dropout(dropout))
    for i in range(1, n_layers):
        return_sequences = (i+1 != n_layers)
        model.add(LSTM(lstmsize,
                       kernel_initializer='glorot_uniform',
                       return_sequences=return_sequences,
                       kernel_regularizer=regularizers.l1_l2(l1,l2),
                       recurrent_regularizer=regularizers.l1_l2(l1,l2)))
        model.add(Dropout(dropout))
    
    model.add(Dense(num_chars, kernel_initializer='glorot_uniform',activation='softmax'))
    opt = Adam(learning_rate=0.005)
    model.compile(loss='categorical_crossentropy', optimizer=opt)
    return model

def get_startend(log): #get first and last activity, we use it to stop the simulation. Should be changed!!!
    return log[0][0], log[0][-1]

def get_probabilities(rnnmodel, xlists,  nr_act, maxlen, prefixlen):
    #assume xlist is a list with the x (prefix) untill now 
    all_x = keras.preprocessing.sequence.pad_sequences(xlists, maxlen=maxlen, padding="pre")
    all_x = all_x[:,-(prefixlen):]
    results = rnnmodel.predict(all_x)
    return results

def normalize(probs): #normalize probabilities to sum to 1
    examplesize, actsize = probs.shape
    newy = []
    for i in range(examplesize):
        normalizer = 1 / float( sum(probs[i]) )
        ynorm = [float(l) * normalizer for l in probs[i]]
        newy.append(ynorm)
    return newy


def choose_act_all(all_y): #randomly choose an activity, stochastically
    #p want a list of probabilities    
    chosen_acts = []
    for i in range(len(all_y)):
        chosen_acts.append(np.random.choice(np.arange(0, len(all_y[i])), p=all_y[i])+1)  
    return(chosen_acts)   # +1 because number encodig starts at 1 not 0

def cut_end(log, endact):
    logsize, tracesize = log.shape
    print(log.shape)
    newlog = []
    for i in range(0, logsize):
        trace = []
        for j in range(0, tracesize):
            if log[i][j] == endact:
                trace.append(log[i][j])
                break
            else:
                trace.append(log[i][j])
        newlog.append(trace)
    return(newlog)


def simulate_log(RNNmodel, logsize, startact, endact, maxlen, mapping, prefixlen): #Use RNN to simulate log
    log = np.zeros((logsize, maxlen+1), int)
    for i in range(0, logsize): #start every trace with the start activity
        log[i][0] = startact
    print(log)
    for j in range(1,maxlen+1): #check if 0 or 1 and ml or ml - 1 #we took 50 for with loops   
        print("finding activity nr", j+1)   
        prefixes = np.array([log[i][0:j] for i in range(0, logsize)])
        print(prefixes)
        probs = get_probabilities(RNNmodel, prefixes, len(mapping), maxlen, prefixlen)
        #we need to do this because otherwise probabilities sum over 1 
        ynorm = normalize(probs) 
        nextacts = choose_act_all(ynorm) 
        for i in range(0, logsize):
            log[i][j] = nextacts[i]
    print(log)
    corrected_log = cut_end(log, endact)      
    return(corrected_log) 

def save_log(loglist, filename): #save a list of lists 
    df = pd.DataFrame.from_records(loglist)
    df.to_csv(filename, index=False)

def number_to_one_hot_X(X, dict_size): #if we want 
    newX = []
    for example in X:
        new_ex = []
        for i in range(len(example)):
            onehot = [0]*dict_size #changed
            if example[i] != 0:
                onehot[example[i] - 1] = 1 #-1 because 
            new_ex.append(onehot)
        newX.append(new_ex)
    return(np.array(newX))


def OHget_probabilities(rnnmodel, xlists,  nr_act, maxlen, prefixlen):
    #assume xlist is a list with the x (prefix) untill now 
    all_x = keras.preprocessing.sequence.pad_sequences(xlists, maxlen=maxlen, padding="pre")
    all_x = all_x[:,-(prefixlen):]
    all_x = number_to_one_hot_X(all_x, nr_act)
    results = rnnmodel.predict(all_x)
    return results




def OHsimulate_log(RNNmodel, logsize, startact, endact, maxlen, mapping, prefixlen): #Use RNN to simulate log
    log = np.zeros((logsize, maxlen+1), int)
    for i in range(0, logsize): #start every trace with the start activity
        log[i][0] = startact
    print(log)
    for j in range(1,maxlen+1): #check if 0 or 1 and ml or ml - 1 #we took 50 for with loops   
        print("finding activity nr", j+1)   
        prefixes = np.array([log[i][0:j] for i in range(0, logsize)])
        print(prefixes)
        probs = OHget_probabilities(RNNmodel, prefixes, len(mapping), maxlen, prefixlen)
        #we need to do this because otherwise probabilities sum over 1 
        ynorm = normalize(probs) 
        nextacts = choose_act_all(ynorm) 
        for i in range(0, logsize):
            log[i][j] = nextacts[i]
    print(log)
    corrected_log = cut_end(log, endact)      
    return(corrected_log) 

def do_grid_search_with_embedding_one_variant(variant, prefixlen):
    trainname = 'Parallel/Training_logs/log_'+ str(variant) + '.csv'
    #reload mapping
    mappingfilename = 'Parallel/mapping.txt' 
    with open(mappingfilename) as f:
        mapping = json.loads(f.read())

        
    train_log = remove_nan(import_log(trainname))
    X_train, y_train = create_XY_prefix(train_log, len(mapping), prefixlen)
    
    
    maxLength = max(map(len, train_log))
    
    grid_nr_layers =  [1, 2]
    grid_layersize = [16, 32, 64]
    grid_dropout = [0.0, 0.2, 0.4]
    grid_reg = [0.0, 0.00001, 0.0001, 0.001, 0.01]
    
    for num_layers in grid_nr_layers:
        for layersize in grid_layersize:
            for dropout in grid_dropout:
                for reg in grid_reg:
                    filename = 'Parallel/Simulated_logs/Var'+str(variant)+'/SIMLOG_NL'+str(num_layers)+'emb'+'Y'+'LS'+str(layersize)+'D'+str(dropout).replace('0.', '')+'R'+str(reg).replace('.', '')+'.csv'
                    if os.path.exists(filename): 
                        print("Already exists:", num_layers, layersize, dropout, reg)
                        continue
                    model = get_model_onedirectional(maxlen=prefixlen, num_chars=len(mapping), embedding=True, 
                                                     n_layers=num_layers, lstmsize=layersize, dropout = dropout, 
                                                     l1=reg, l2=reg)
                    model.summary()
                    batch_size = 128
                    early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
                    lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', 
                                                   min_delta=0.0001, cooldown=0, min_lr=0)

                    # train the model, output generated text after each iteration
                    history = model.fit(X_train, y_train, 
                                        validation_split=0.2,
                                        callbacks=[early_stopping, lr_reducer],
                                        batch_size=batch_size, epochs=600, verbose=2)
                    
                    
                    #save RNN model in case we ever need it
                    modelfilename = 'Parallel/Models/Var'+str(variant)+'/model_NL'+str(num_layers)+'emb'+'Y'+'LS'+str(layersize)+'D'+str(dropout).replace('0.', '')+'R'+str(reg).replace('.', '')
                    model.save(modelfilename)
                    
                    start,end = get_startend(train_log)
                    
                    #CHANGE THIS BELOW!!!!!
                    
                    
                    simlog = simulate_log(model, 12000, start, end, maxLength-1, mapping, prefixlen)
                    
                    #save simulated log
                    SimLogName = 'Parallel/Simulated_logs/Var'+str(variant)+'/SIMLOG_NL'+str(num_layers)+'emb'+'Y'+'LS'+str(layersize)+'D'+str(dropout).replace('0.', '')+'R'+str(reg).replace('.', '')+'.csv'
                    save_log(simlog, SimLogName)

    
def do_grid_search_without_embedding_one_variant(variant, prefixlen):
    trainname = 'Parallel/Training_logs/log_'+ str(variant) + '.csv'
    #reload mapping
    mappingfilename = 'Parallel/mapping.txt' 
    with open(mappingfilename) as f:
        mapping = json.loads(f.read())

        
    train_log = remove_nan(import_log(trainname))
    X_train, y_train = create_XY_prefix(train_log, len(mapping), prefixlen)
    
    X_train = number_to_one_hot_X(X_train, len(mapping))
    
    maxLength = max(map(len, train_log))
    
    grid_nr_layers =  [1,2]
    grid_layersize = [16, 32, 64]
    grid_dropout = [0.0, 0.2, 0.4]
    grid_reg = [0.0, 0.00001, 0.0001, 0.001, 0.01]
    
    for num_layers in grid_nr_layers:
        for layersize in grid_layersize:
            for dropout in grid_dropout:
                for reg in grid_reg:
                    filename = 'Parallel/Simulated_logs/Var'+str(variant)+'/SIMLOG_NL'+str(num_layers)+'emb'+'N'+'LS'+str(layersize)+'D'+str(dropout).replace('0.', '')+'R'+str(reg).replace('.', '')+'.csv'
                    if os.path.exists(filename): 
                        print("Already exists:", num_layers, layersize, dropout, reg)
                        continue
                    model = get_model_onedirectional(maxlen=prefixlen, num_chars=len(mapping), embedding=False, 
                                                     n_layers=num_layers, lstmsize=layersize, dropout = dropout, 
                                                     l1=reg, l2=reg)
                    model.summary()
                    batch_size = 128
                    early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
                    lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', 
                                                   min_delta=0.0001, cooldown=0, min_lr=0)

                    # train the model, output generated text after each iteration
                    history = model.fit(X_train, y_train, 
                                        validation_split=0.2,
                                        callbacks=[early_stopping, lr_reducer],
                                        batch_size=batch_size, epochs=600, verbose=2)
                    
                    
                    #save RNN model in case we ever need it
                    modelfilename = 'Parallel/Models/Var'+str(variant)+'/model_NL'+str(num_layers)+'emb'+'N'+'LS'+str(layersize)+'D'+str(dropout).replace('0.', '')+'R'+str(reg).replace('.', '')
                    model.save(modelfilename)
                    
                    start,end = get_startend(train_log)
                    
                    #CHANGE THIS BELOW!!!!!
                    
                    
                    simlog = OHsimulate_log(model, 12000, start, end, maxLength-1, mapping, prefixlen)
                    
                    #save simulated log
                    SimLogName = 'Parallel/Simulated_logs/Var'+str(variant)+'/SIMLOG_NL'+str(num_layers)+'emb'+'N'+'LS'+str(layersize)+'D'+str(dropout).replace('0.', '')+'R'+str(reg).replace('.', '')+'.csv'
                    save_log(simlog, SimLogName)

    
    

In [17]:
do_grid_search_with_embedding_one_variant(1, 10)

Already exists: 1 16 0.0 0.0
Already exists: 1 16 0.0 1e-05
Already exists: 1 16 0.0 0.0001
Already exists: 1 16 0.0 0.001
Already exists: 1 16 0.0 0.01
Already exists: 1 16 0.2 0.0
Already exists: 1 16 0.2 1e-05
Already exists: 1 16 0.2 0.0001
Already exists: 1 16 0.2 0.001
Already exists: 1 16 0.2 0.01
Already exists: 1 16 0.4 0.0
Already exists: 1 16 0.4 1e-05
Already exists: 1 16 0.4 0.0001
Already exists: 1 16 0.4 0.001
Already exists: 1 16 0.4 0.01
Already exists: 1 32 0.0 0.0
Already exists: 1 32 0.0 1e-05
Already exists: 1 32 0.0 0.0001
Already exists: 1 32 0.0 0.001
Already exists: 1 32 0.0 0.01
Already exists: 1 32 0.2 0.0
Already exists: 1 32 0.2 1e-05
Already exists: 1 32 0.2 0.0001
Already exists: 1 32 0.2 0.001
Already exists: 1 32 0.2 0.01
Already exists: 1 32 0.4 0.0
Already exists: 1 32 0.4 1e-05
Already exists: 1 32 0.4 0.0001
Already exists: 1 32 0.4 0.001
Already exists: 1 32 0.4 0.01
Already exists: 1 64 0.0 0.0
Already exists: 1 64 0.0 1e-05
Already exists: 1 64 0

In [18]:
do_grid_search_without_embedding_one_variant(1, 10)

Already exists: 1 16 0.0 0.0
Already exists: 1 16 0.0 1e-05
Already exists: 1 16 0.0 0.0001
Already exists: 1 16 0.0 0.001
Already exists: 1 16 0.0 0.01
Already exists: 1 16 0.2 0.0
Already exists: 1 16 0.2 1e-05
Already exists: 1 16 0.2 0.0001
Already exists: 1 16 0.2 0.001
Already exists: 1 16 0.2 0.01
Already exists: 1 16 0.4 0.0
Already exists: 1 16 0.4 1e-05
Already exists: 1 16 0.4 0.0001
Already exists: 1 16 0.4 0.001
Already exists: 1 16 0.4 0.01
Already exists: 1 32 0.0 0.0
Already exists: 1 32 0.0 1e-05
Already exists: 1 32 0.0 0.0001
Already exists: 1 32 0.0 0.001
Already exists: 1 32 0.0 0.01
Already exists: 1 32 0.2 0.0
Already exists: 1 32 0.2 1e-05
Already exists: 1 32 0.2 0.0001
Already exists: 1 32 0.2 0.001
Already exists: 1 32 0.2 0.01
Already exists: 1 32 0.4 0.0
Already exists: 1 32 0.4 1e-05
Already exists: 1 32 0.4 0.0001
Already exists: 1 32 0.4 0.001
Already exists: 1 32 0.4 0.01
Already exists: 1 64 0.0 0.0
Already exists: 1 64 0.0 1e-05
Already exists: 1 64 0

Epoch 106/600
894/894 - 6s - loss: 0.4240 - val_loss: 0.4196
Epoch 107/600
894/894 - 6s - loss: 0.4246 - val_loss: 0.4170
Epoch 108/600
894/894 - 5s - loss: 0.4242 - val_loss: 0.4182
Epoch 109/600
894/894 - 7s - loss: 0.4245 - val_loss: 0.4207
Epoch 110/600
894/894 - 6s - loss: 0.4241 - val_loss: 0.4166
Epoch 111/600
894/894 - 5s - loss: 0.4237 - val_loss: 0.4183
Epoch 112/600
894/894 - 5s - loss: 0.4238 - val_loss: 0.4152
Epoch 113/600
894/894 - 5s - loss: 0.4237 - val_loss: 0.4155
Epoch 114/600
894/894 - 5s - loss: 0.4242 - val_loss: 0.4149
Epoch 115/600
894/894 - 5s - loss: 0.4233 - val_loss: 0.4169
Epoch 116/600
894/894 - 5s - loss: 0.4235 - val_loss: 0.4171
Epoch 117/600
894/894 - 6s - loss: 0.4241 - val_loss: 0.4161
Epoch 118/600
894/894 - 6s - loss: 0.4231 - val_loss: 0.4178
Epoch 119/600
894/894 - 5s - loss: 0.4238 - val_loss: 0.4163
Epoch 120/600
894/894 - 5s - loss: 0.4239 - val_loss: 0.4182
Epoch 121/600
894/894 - 5s - loss: 0.4241 - val_loss: 0.4150
Epoch 122/600
894/894 - 

Epoch 241/600
894/894 - 5s - loss: 0.4117 - val_loss: 0.4087
Epoch 242/600
894/894 - 5s - loss: 0.4116 - val_loss: 0.4079
Epoch 243/600
894/894 - 5s - loss: 0.4119 - val_loss: 0.4082
Epoch 244/600
894/894 - 5s - loss: 0.4116 - val_loss: 0.4080
Epoch 245/600
894/894 - 5s - loss: 0.4117 - val_loss: 0.4081
Epoch 246/600
894/894 - 5s - loss: 0.4117 - val_loss: 0.4082
Epoch 247/600
894/894 - 5s - loss: 0.4116 - val_loss: 0.4079
Epoch 248/600
894/894 - 5s - loss: 0.4117 - val_loss: 0.4079
Epoch 249/600
894/894 - 5s - loss: 0.4117 - val_loss: 0.4079
Epoch 250/600
894/894 - 5s - loss: 0.4119 - val_loss: 0.4079
Epoch 251/600
894/894 - 5s - loss: 0.4115 - val_loss: 0.4080
Epoch 252/600
894/894 - 5s - loss: 0.4117 - val_loss: 0.4079
Epoch 253/600
894/894 - 5s - loss: 0.4116 - val_loss: 0.4078
Epoch 254/600
894/894 - 5s - loss: 0.4112 - val_loss: 0.4079
Epoch 255/600
894/894 - 5s - loss: 0.4114 - val_loss: 0.4079
Epoch 256/600
894/894 - 5s - loss: 0.4116 - val_loss: 0.4077
Epoch 257/600
894/894 - 

INFO:tensorflow:Assets written to: Parallel/Models/Var1/model_NL1embNLS64D2R0001\assets


INFO:tensorflow:Assets written to: Parallel/Models/Var1/model_NL1embNLS64D2R0001\assets


...

894/894 - 5s - loss: 0.6609 - val_loss: 0.5787
Epoch 93/600
894/894 - 5s - loss: 0.6586 - val_loss: 0.5759
Epoch 94/600
894/894 - 5s - loss: 0.6581 - val_loss: 0.5786
Epoch 95/600
894/894 - 6s - loss: 0.6602 - val_loss: 0.5733
Epoch 96/600
894/894 - 6s - loss: 0.6572 - val_loss: 0.5768
Epoch 97/600
894/894 - 6s - loss: 0.6575 - val_loss: 0.5779
Epoch 98/600
894/894 - 6s - loss: 0.6564 - val_loss: 0.5765
Epoch 99/600
894/894 - 5s - loss: 0.6567 - val_loss: 0.5705
Epoch 100/600
894/894 - 5s - loss: 0.6563 - val_loss: 0.5781
Epoch 101/600
894/894 - 5s - loss: 0.6560 - val_loss: 0.5741
Epoch 102/600
894/894 - 6s - loss: 0.6563 - val_loss: 0.5737
Epoch 103/600
894/894 - 6s - loss: 0.6569 - val_loss: 0.5793
Epoch 104/600
894/894 - 6s - loss: 0.6560 - val_loss: 0.5733
Epoch 105/600
894/894 - 6s - loss: 0.6572 - val_loss: 0.5664
Epoch 106/600
894/894 - 5s - loss: 0.6565 - val_loss: 0.5736
Epoch 107/600
894/894 - 5s - loss: 0.6566 - val_loss: 0.5788
Epoch 108/600
894/894 - 5s - loss: 0.6558 - v

Epoch 227/600
894/894 - 5s - loss: 0.6143 - val_loss: 0.5353
Epoch 228/600
894/894 - 5s - loss: 0.6154 - val_loss: 0.5369
Epoch 229/600
894/894 - 5s - loss: 0.6149 - val_loss: 0.5354
Epoch 230/600
894/894 - 6s - loss: 0.6144 - val_loss: 0.5363
Epoch 231/600
894/894 - 5s - loss: 0.6141 - val_loss: 0.5360
Epoch 232/600
894/894 - 7s - loss: 0.6143 - val_loss: 0.5378
Epoch 233/600
894/894 - 6s - loss: 0.6152 - val_loss: 0.5390
Epoch 234/600
894/894 - 6s - loss: 0.6142 - val_loss: 0.5362
Epoch 235/600
894/894 - 7s - loss: 0.6149 - val_loss: 0.5361
Epoch 236/600
894/894 - 8s - loss: 0.6124 - val_loss: 0.5338
Epoch 237/600
894/894 - 6s - loss: 0.6113 - val_loss: 0.5324
Epoch 238/600
894/894 - 6s - loss: 0.6115 - val_loss: 0.5340
Epoch 239/600
894/894 - 6s - loss: 0.6110 - val_loss: 0.5363
Epoch 240/600
894/894 - 5s - loss: 0.6117 - val_loss: 0.5339
Epoch 241/600
894/894 - 5s - loss: 0.6145 - val_loss: 0.5339
Epoch 242/600
894/894 - 5s - loss: 0.6106 - val_loss: 0.5326
Epoch 243/600
894/894 - 

INFO:tensorflow:Assets written to: Parallel/Models/Var1/model_NL1embNLS64D4R001\assets


INFO:tensorflow:Assets written to: Parallel/Models/Var1/model_NL1embNLS64D4R001\assets


[[6 0 0 ... 0 0 0]
 [6 0 0 ... 0 0 0]
 [6 0 0 ... 0 0 0]
 ...
 [6 0 0 ... 0 0 0]
 [6 0 0 ... 0 0 0]
 [6 0 0 ... 0 0 0]]
finding activity nr 2
[[6]
 [6]
 [6]
 ...
 [6]
 [6]
 [6]]
finding activity nr 3
[[6 7]
 [6 7]
 [6 7]
 ...
 [6 7]
 [6 7]
 [6 7]]
finding activity nr 4
[[6 7 8]
 [6 7 8]
 [6 7 8]
 ...
 [6 7 8]
 [6 7 8]
 [6 7 8]]
finding activity nr 5
[[ 6  7  8 13]
 [ 6  7  8 13]
 [ 6  7  8 13]
 ...
 [ 6  7  8 13]
 [ 6  7  8 13]
 [ 6  7  8 13]]
finding activity nr 6
[[ 6  7  8 13 11]
 [ 6  7  8 13 11]
 [ 6  7  8 13  1]
 ...
 [ 6  7  8 13  1]
 [ 6  7  8 13  4]
 [ 6  7  8 13  4]]
finding activity nr 7
[[ 6  7  8 13 11  2]
 [ 6  7  8 13 11  3]
 [ 6  7  8 13  1 11]
 ...
 [ 6  7  8 13  1  3]
 [ 6  7  8 13  4  4]
 [ 6  7  8 13  4  3]]
finding activity nr 8
[[ 6  7  8 ... 11  2  1]
 [ 6  7  8 ... 11  3  2]
 [ 6  7  8 ...  1 11  4]
 ...
 [ 6  7  8 ...  1  3  2]
 [ 6  7  8 ...  4  4  2]
 [ 6  7  8 ...  4  3  3]]
finding activity nr 9
[[ 6  7  8 ...  2  1  4]
 [ 6  7  8 ...  3  2  1]
 [ 6  7  8 .